<a href="https://colab.research.google.com/github/Mariihmp/RL_Notebooks/blob/main/TD(lambda)_backwardView.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
import numpy as np
from google.colab import drive
drive.mount('/content/drive')
project_folder_path = '/content/drive/MyDrive/session-2'
sys.path.append(project_folder_path)

Mounted at /content/drive


In [ ]:
from GridWorld_env import GridWorld
env = GridWorld()


In [ ]:
def td_lambda_evaluation(env, policy_to_evaluate, num_episodes, alpha=0.1, lambda_param=0.7):

    V = np.zeros(env.num_states)

    print(f"\nStarting TD(lambda) Policy Evaluation for {num_episodes} episodes")
    print(f"Alpha={alpha}, Lambda={lambda_param}")

    for i_episode in range(num_episodes):
        if (i_episode + 1) % (num_episodes // 10 if num_episodes >=10 else 1) == 0:
            print(f"  Episode {i_episode + 1}/{num_episodes}")

        eligibility_traces = np.zeros(env.num_states)
        state_idx = env.reset()
        done = False
        max_steps = 100
        steps = 0

        while not done and steps < max_steps:
            action_idx = np.random.choice(env.num_actions,
                                          p=policy_to_evaluate[state_idx])

            next_state_idx, reward, done = env.step(state_idx,
                                                    action_idx)

            #if terminal, V(next_state_idx) is 0
            td_target = reward + (env.gamma * V[next_state_idx] if not done else 0)
            td_error = td_target - V[state_idx]

            eligibility_traces[state_idx] += 1

            eligibility_traces = env.gamma * lambda_param * eligibility_traces
            eligibility_traces[state_idx] += 1


            eligibility_traces[state_idx] += 1 # Mark presence

            # Update V for all states
            for s_trace_idx in range(env.num_states):
                V[s_trace_idx] += alpha * td_error * eligibility_traces[s_trace_idx]
                if env.idx_to_state[s_trace_idx] == env.goal_state:
                    V[s_trace_idx] = 0

            # Decay traces
            eligibility_traces *= env.gamma * lambda_param


            state_idx = next_state_idx
            steps +=1
            if env.idx_to_state[state_idx] == env.goal_state:
                V[state_idx] = 0

    return V


In [ ]:
# Let's evaluate the optimal policy found by Value Iteration
policy_from_vi = optimal_policy_vi # This is already P(a|s)

num_td_episodes = 2000
alpha_td = 0.1
lambda_val = 0.9 # A common choice for lambda

print("\n--- Evaluating V(s) for the Policy from Value Iteration using TD(lambda) ---")
V_td_lambda = td_lambda_evaluation(env, policy_from_vi, num_td_episodes, alpha=alpha_td, lambda_param=lambda_val)

print("\nV(s) estimated by TD(lambda):")
for s_idx, v_val in enumerate(V_td_lambda):
    print(f"  State {env.idx_to_state[s_idx]}: {v_val:.2f}")

print("\nFor comparison, V(s) from Value Iteration:")
for s_idx, v_val in enumerate(V_optimal_vi):
    print(f"  State {env.idx_to_state[s_idx]}: {v_val:.2f}")

NameError: name 'optimal_policy_vi' is not defined